<a href="https://colab.research.google.com/github/AlinaStudyPM/UniversityML/blob/main/MyML_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Считываем данные + удаляем лишние строки + добавляем столбец class
2) Разделяем на три датасета по материалу
3) Общая функция
  1) Выделяем X и Y
  2) Разделяем на обучающую и тестовую выборки
  3) Нормируем значения?
  4) Создаём модель knn и обучаем на данных
  5) Предсказываем Y для тестовых данных и сверяем с истинными значениями
  6) Выводим информацию об оценке качества классификации




In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
data = pd.read_excel('14 CERAMICA_DATA.xlsx')
data = data[data['obrazec'] != 'obrazec']
data['class'] = data['obrazec'].apply(lambda x: 'str' if 'str' in x else 'neth')
data

,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr,class
0,str_5_mid,1_massa,80,281,1394,153,98,4480,97,25,24,45,169,174,50,98,158,10,41,str
1,str_5_mid,3_white,172,268,202,86,90,215,93,99,161,127,18552,14319,179,101,120,274,84,str
2,str_5_mid,6_blue,172,344,290,115,103,418,296,205,190,174,21025,15871,198,101,107,302,86,str
3,str_5_big,1_massa,90,285,2137,182,127,4807,85,71,36,47,235,219,38,82,189,8,47,str
4,str_5_big,3_massa,88,281,2000,169,91,3827,96,151,15,56,513,445,33,69,180,9,64,str
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,rot_9,blue,137,377,341,86,100,1105,1455,1359,156,115,16321,10139,504,120,140,292,78,neth
172,rot_9,massa,52,122,1547,160,78,3850,59,22,38,30,310,285,47,80,215,6,37,neth
173,rot_10,white,150,259,422,89,90,269,73,100,111,109,15691,11627,185,106,130,369,63,neth
174,rot_10,blue,158,301,312,81,101,1417,1157,642,162,137,19475,11575,630,101,141,316,107,neth


In [4]:
data_mass = data[data['point'].str.contains('massa')]
data_mass

,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr,class
0,str_5_mid,1_massa,80,281,1394,153,98,4480,97,25,24,45,169,174,50,98,158,10,41,str
3,str_5_big,1_massa,90,285,2137,182,127,4807,85,71,36,47,235,219,38,82,189,8,47,str
4,str_5_big,3_massa,88,281,2000,169,91,3827,96,151,15,56,513,445,33,69,180,9,64,str
9,amst_1_sm,1_massa,87,201,3086,140,112,4525,80,32,39,43,526,460,48,80,150,12,50,neth
10,amst_1_sm,2_massa,68,149,2121,112,65,3191,51,25,32,52,477,387,46,76,186,11,41,neth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,rot_6,massa,75,187,1924,189,163,4930,219,28,30,46,530,438,46,64,217,9,43,neth
166,rot_7,massa,45,104,2546,62,55,2204,77,14,21,27,197,187,35,51,119,4,27,neth
169,rot_8,massa,75,170,2073,117,97,3877,53,23,30,41,423,331,48,83,229,5,40,neth
172,rot_9,massa,52,122,1547,160,78,3850,59,22,38,30,310,285,47,80,215,6,37,neth


In [5]:
data_white = data[data['point'].str.contains('white')]
data_white

,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr,class
1,str_5_mid,3_white,172,268,202,86,90,215,93,99,161,127,18552,14319,179,101,120,274,84,str
5,str_5_big,4_white,156,182,241,91,88,279,94,107,202,171,20230,15106,166,105,115,299,85,str
6,str_5_big,5_white,178,239,242,115,84,271,112,96,204,162,20030,15566,172,101,110,291,89,str
11,amst_1_sm,3_white,143,318,397,79,86,258,79,92,154,100,14825,11357,163,98,130,402,68,neth
12,amst_1_sm,4_white,142,366,428,100,96,343,77,126,184,155,17048,13286,146,118,164,421,74,neth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,rot_6,white,141,291,457,86,88,234,75,68,158,108,15890,12149,154,121,120,349,61,neth
164,rot_7,white,116,293,377,74,82,230,68,86,127,107,14407,10934,143,84,123,359,64,neth
167,rot_8,white,131,269,451,75,70,265,73,103,167,111,14275,11138,146,103,117,374,71,neth
170,rot_9,white,129,318,392,84,69,260,60,73,110,119,13381,10611,129,122,127,344,63,neth


In [6]:
data_blue = data[data['point'].str.contains('blue')]
data_blue

,obrazec,point,Si,K,Ca,Ti,Mn,Fe,Co,Ni,Cu,Zn,Pb1,Pb2,As2,Rb,Sr,Sn,Cr,class
2,str_5_mid,6_blue,172,344,290,115,103,418,296,205,190,174,21025,15871,198,101,107,302,86,str
7,str_5_big,6_blue,156,221,236,113,107,525,551,272,311,161,21024,14888,372,84,105,246,90,str
8,str_5_big,8_blue,165,209,205,123,100,563,609,309,289,145,20575,15012,261,116,113,250,86,str
13,amst_1_sm,5_blue,145,286,375,84,109,729,578,453,176,132,18862,11734,586,111,135,342,88,neth
14,amst_1_sm,6_blue,157,265,443,91,107,659,427,264,159,127,17635,12083,413,101,147,352,86,neth
19,amst_1_big,5_blue,153,312,335,82,99,934,816,559,154,125,16848,10608,564,110,145,295,76,neth
20,amst_1_big,6_blue,162,365,294,95,114,1263,1107,884,125,147,18099,11249,614,95,127,256,88,neth
23,amst_2_sm,5_blue,137,313,287,83,107,1015,890,647,160,135,18106,10972,603,95,127,295,80,neth
26,amst_2_big,5_blue,158,348,321,91,110,976,979,716,159,141,18282,10593,649,130,137,325,80,neth
28,rot_3_big,3_blue,142,322,316,74,105,1352,1232,680,154,117,18050,10980,560,117,126,295,79,neth


In [17]:
def Solution(data):
  #выделяем независимые переменные и зависимую переменную
  X = data[['Si',	'K',	'Ca',	'Ti',	'Mn',	'Fe',	'Co',	'Ni',	'Cu',	'Zn',	'Pb1',	'Pb2',	'As2',	'Rb',	'Sr',	'Sn',	'Cr']]
  y = data['class']

  #разделяем на обучающую и тестовую выборки
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  #Стандартизация (X - ср.знач.) / станд.откл.
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  #Создаём модель knn и обучаем её
  k = 3
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)

  #Применяем к тестовым данным
  y_pred = knn.predict(X_test)

  #Выводим результаты
  print("Predictions:", y_pred)
  print("Actual:", y_test.values)
  '''
  test_data = data.loc[X_test.index]
  results_data = pd.DataFrame({
        'obrazec': test_data['obrazec'].values,
        'point': test_data['point'].values,
        'Predictions': y_pred,
        'Actual': y_test.values
    })
  # Выводим таблицу с результатами
  print(results_data)
  '''

  #Оценка качества классификации
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')  # 'weighted' учитывает доли классов
  recall = recall_score(y_test, y_pred, average='weighted')
  f1 = f1_score(y_test, y_pred, average='weighted')

    # Выводим оценки
  print(f'Accuracy: {accuracy * 100:.2f}%')
  print(f'Precision: {precision * 100:.2f}%')
  print(f'Recall: {recall * 100:.2f}%')
  print(f'F1 Score: {f1 * 100:.2f}%')

In [18]:
Solution(data_mass)

Predictions: ['neth' 'neth' 'str' 'neth' 'neth' 'str' 'neth' 'neth' 'str' 'neth' 'neth'
 'neth' 'neth' 'str' 'str' 'str' 'neth' 'neth' 'neth']
Actual: ['neth' 'neth' 'str' 'neth' 'neth' 'str' 'neth' 'neth' 'str' 'neth' 'neth'
 'neth' 'neth' 'neth' 'str' 'neth' 'neth' 'neth' 'neth']
Accuracy: 89.47%
Precision: 92.98%
Recall: 89.47%
F1 Score: 90.15%


In [19]:
Solution(data_white)

Predictions: ['str' 'neth' 'neth' 'str' 'str' 'neth' 'str' 'neth' 'neth' 'neth' 'str'
 'neth' 'neth' 'neth' 'neth' 'neth' 'neth' 'neth' 'neth']
Actual: ['str' 'neth' 'neth' 'str' 'neth' 'neth' 'str' 'neth' 'neth' 'neth' 'str'
 'neth' 'neth' 'neth' 'neth' 'neth' 'neth' 'neth' 'neth']
Accuracy: 94.74%
Precision: 95.79%
Recall: 94.74%
F1 Score: 94.94%


In [20]:
Solution(data_blue)

Predictions: ['str' 'neth' 'neth' 'str' 'neth' 'neth' 'neth' 'str' 'neth' 'neth' 'neth'
 'neth' 'neth' 'str' 'neth' 'neth' 'neth']
Actual: ['str' 'neth' 'neth' 'neth' 'neth' 'neth' 'neth' 'str' 'neth' 'neth'
 'neth' 'neth' 'neth' 'str' 'neth' 'neth' 'neth']
Accuracy: 94.12%
Precision: 95.59%
Recall: 94.12%
F1 Score: 94.43%


Старый код

Дополнительные блоки кода

In [7]:
unique_obrazec = data['obrazec'].unique()
unique_obrazecm

array(['str_5_mid', 'str_5_big', 'amst_1_sm', 'amst_1_big', 'amst_2_sm',
       'amst_2_big', 'rot_3_big', 'rot_3_sm', 'utr_4_big', 'srt_5_sm',
       'rot_mdk406', 'rot_acp63', 'urt_mgk_g_326', 'urt_mgk_g_319',
       'holl_mdk_n_718', 'holl_acp_176', 'amst_acp_61', 'str_6', 'str_7',
       'str_8', 'str_асп_1849', 'str_асп_1850', 'str_асп_1847',
       'str_асп_1848', 'str_асп_1846', 'str_асп_1844', 'str_асп_1845',
       'str_acп_1851', 'str_50', 'str_асп_1910', 'amst_1', 'amst_1a',
       'amst_2', 'amst_3', 'amst_4', 'amst_5', 'amst_6', 'amst_7',
       'amst_8', 'amst_8a', 'amst_11', 'amst_12', 'amst_9', 'amst_10',
       'amst_13', 'amst_14', 'amst_15', 'rot_4', 'rot_5', 'rot_6',
       'rot_7', 'rot_8', 'rot_9', 'rot_10'], dtype=object)